In [1]:
import pandas as pd
import json
import numpy as np
from typing import Union
import os

def process_empty_qualitative_factors(
    csv_path: str,
    output_path: str = None,
    replace_with_nan: bool = True
) -> pd.DataFrame:
    """
    정성적 요인 컬럼에서 빈 배열을 NaN 또는 None으로 처리

    Args:
        csv_path: 입력 CSV 파일 경로
        output_path: 출력 CSV 파일 경로 (None이면 원본 파일명에 _processed 추가)
        replace_with_nan: True면 NaN, False면 None으로 처리

    Returns:
        처리된 DataFrame
    """

    # CSV 파일 로드
    print("CSV 파일 로딩 중...")
    df = pd.read_csv(csv_path)

    # 출력 경로 설정
    if output_path is None:
        base_path = os.path.splitext(csv_path)[0]
        output_path = f"{base_path}_processed.csv"

    # 빈 배열 개수 카운트 (처리 전)
    positive_empty_count = 0
    negative_empty_count = 0

    print("빈 배열 검사 및 처리 중...")

    # positive_factors 처리
    for idx, value in df['positive_factors'].items():
        if pd.isna(value) or value == '' or value == '[]':
            positive_empty_count += 1
            df.at[idx, 'positive_factors'] = np.nan if replace_with_nan else None
        else:
            try:
                # JSON 문자열을 파싱하여 빈 배열인지 확인
                parsed = json.loads(value) if isinstance(value, str) else value
                if isinstance(parsed, list) and len(parsed) == 0:
                    positive_empty_count += 1
                    df.at[idx, 'positive_factors'] = np.nan if replace_with_nan else None
            except (json.JSONDecodeError, TypeError):
                # JSON 파싱 실패시 원본 유지
                pass

    # negative_factors 처리
    for idx, value in df['negative_factors'].items():
        if pd.isna(value) or value == '' or value == '[]':
            negative_empty_count += 1
            df.at[idx, 'negative_factors'] = np.nan if replace_with_nan else None
        else:
            try:
                # JSON 문자열을 파싱하여 빈 배열인지 확인
                parsed = json.loads(value) if isinstance(value, str) else value
                if isinstance(parsed, list) and len(parsed) == 0:
                    negative_empty_count += 1
                    df.at[idx, 'negative_factors'] = np.nan if replace_with_nan else None
            except (json.JSONDecodeError, TypeError):
                # JSON 파싱 실패시 원본 유지
                pass

    # 결과 저장
    df.to_csv(output_path, index=False, encoding='utf-8')

    # 처리 결과 출력
    print(f"\n=== 처리 결과 ===")
    print(f"총 행 수: {len(df):,}")
    print(f"빈 positive_factors 처리: {positive_empty_count:,}개")
    print(f"빈 negative_factors 처리: {negative_empty_count:,}개")
    print(f"처리 후 positive_factors NaN 비율: {df['positive_factors'].isna().sum() / len(df) * 100:.1f}%")
    print(f"처리 후 negative_factors NaN 비율: {df['negative_factors'].isna().sum() / len(df) * 100:.1f}%")
    print(f"처리된 파일 저장: {output_path}")

    return df

def analyze_qualitative_data_quality(df: pd.DataFrame) -> dict:
    """
    정성적 데이터의 품질을 분석

    Args:
        df: 분석할 DataFrame

    Returns:
        분석 결과 딕셔너리
    """

    analysis = {
        'total_records': len(df),
        'positive_factors': {},
        'negative_factors': {}
    }

    for col_name in ['positive_factors', 'negative_factors']:
        col = df[col_name]

        # 기본 통계
        null_count = col.isna().sum()
        valid_count = len(df) - null_count

        # 유효한 데이터에서 요인 개수 분석
        factor_counts = []
        for value in col.dropna():
            if isinstance(value, str) and value.strip():
                try:
                    parsed = json.loads(value)
                    if isinstance(parsed, list):
                        factor_counts.append(len(parsed))
                except:
                    pass

        analysis[col_name] = {
            'null_count': int(null_count),
            'valid_count': int(valid_count),
            'null_percentage': round(null_count / len(df) * 100, 2),
            'valid_percentage': round(valid_count / len(df) * 100, 2),
            'avg_factors_per_record': round(np.mean(factor_counts), 2) if factor_counts else 0,
            'max_factors_per_record': max(factor_counts) if factor_counts else 0,
            'min_factors_per_record': min(factor_counts) if factor_counts else 0
        }

    return analysis

def sample_qualitative_data(df: pd.DataFrame, n_samples: int = 10) -> pd.DataFrame:
    """
    정성적 데이터 샘플을 추출하여 품질 확인

    Args:
        df: DataFrame
        n_samples: 샘플 개수

    Returns:
        샘플 DataFrame
    """

    # 유효한 정성적 데이터가 있는 행들만 필터링
    valid_rows = df[
        (df['positive_factors'].notna()) &
        (df['negative_factors'].notna()) &
        (df['positive_factors'] != '[]') &
        (df['negative_factors'] != '[]')
    ]

    if len(valid_rows) == 0:
        print("유효한 정성적 데이터가 있는 행이 없습니다.")
        return pd.DataFrame()

    # 랜덤 샘플링
    sample_df = valid_rows.sample(min(n_samples, len(valid_rows)))

    # 관련 컬럼만 선택
    columns_to_show = ['corp_code', 'corp_name', 'positive_factors', 'negative_factors']
    available_columns = [col for col in columns_to_show if col in sample_df.columns]

    return sample_df[available_columns]

# 실행 함수
def main():
    """
    메인 실행 함수
    """

    # 파일 경로 설정
    input_file = "D:/JetBrains/ai/add-data-real-company/integrated_financial_qualitative_data.csv"
    output_file = "D:/JetBrains/ai/add-data-real-company/integrated_financial_qualitative_data_processed.csv"

    try:
        # 1. 빈 배열을 NaN으로 처리
        print("=== 빈 배열 처리 시작 ===")
        df_processed = process_empty_qualitative_factors(input_file, output_file)

        # 2. 데이터 품질 분석
        print("\n=== 데이터 품질 분석 ===")
        quality_analysis = analyze_qualitative_data_quality(df_processed)

        print(f"총 레코드 수: {quality_analysis['total_records']:,}")
        print("\n[Positive Factors 분석]")
        pos_stats = quality_analysis['positive_factors']
        print(f"  - 유효 데이터: {pos_stats['valid_count']:,}개 ({pos_stats['valid_percentage']}%)")
        print(f"  - NaN 데이터: {pos_stats['null_count']:,}개 ({pos_stats['null_percentage']}%)")
        print(f"  - 평균 요인 수: {pos_stats['avg_factors_per_record']}개")

        print("\n[Negative Factors 분석]")
        neg_stats = quality_analysis['negative_factors']
        print(f"  - 유효 데이터: {neg_stats['valid_count']:,}개 ({neg_stats['valid_percentage']}%)")
        print(f"  - NaN 데이터: {neg_stats['null_count']:,}개 ({neg_stats['null_percentage']}%)")
        print(f"  - 평균 요인 수: {neg_stats['avg_factors_per_record']}개")

        # 3. 샘플 데이터 확인
        print("\n=== 샘플 데이터 (정성적 요인이 있는 기업들) ===")
        sample_data = sample_qualitative_data(df_processed, 5)

        if not sample_data.empty:
            for idx, row in sample_data.iterrows():
                print(f"\n기업코드: {row.get('corp_code', 'N/A')}")
                if 'corp_name' in row:
                    print(f"기업명: {row['corp_name']}")

                # positive_factors 출력
                try:
                    pos_factors = json.loads(row['positive_factors']) if isinstance(row['positive_factors'], str) else []
                    print(f"긍정 요인 ({len(pos_factors)}개): {pos_factors}")
                except:
                    print(f"긍정 요인: {row['positive_factors']}")

                # negative_factors 출력
                try:
                    neg_factors = json.loads(row['negative_factors']) if isinstance(row['negative_factors'], str) else []
                    print(f"부정 요인 ({len(neg_factors)}개): {neg_factors}")
                except:
                    print(f"부정 요인: {row['negative_factors']}")

        print(f"\n✅ 처리 완료! 결과 파일: {output_file}")

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        raise

if __name__ == "__main__":
    main()

=== 빈 배열 처리 시작 ===
CSV 파일 로딩 중...
빈 배열 검사 및 처리 중...

=== 처리 결과 ===
총 행 수: 2,059
빈 positive_factors 처리: 1,072개
빈 negative_factors 처리: 245개
처리 후 positive_factors NaN 비율: 52.1%
처리 후 negative_factors NaN 비율: 11.9%
처리된 파일 저장: D:/JetBrains/ai/add-data-real-company/integrated_financial_qualitative_data_processed.csv

=== 데이터 품질 분석 ===
총 레코드 수: 2,059

[Positive Factors 분석]
  - 유효 데이터: 987개 (47.94%)
  - NaN 데이터: 1,072개 (52.06%)
  - 평균 요인 수: 2.48개

[Negative Factors 분석]
  - 유효 데이터: 1,814개 (88.1%)
  - NaN 데이터: 245개 (11.9%)
  - 평균 요인 수: 2.38개

=== 샘플 데이터 (정성적 요인이 있는 기업들) ===

기업코드: 204320.0
기업명: HL만도
긍정 요인 (2개): ['HL만도는 2014년 HL 홀딩스로부터 인적 분할된 이후 안정적인 재무제표를 유지하고 있으며, 삼일회계법인에서 감사받은 신뢰성 있는 재무정보를 보유하고 있다.', '자동차 부품 시장에서의 위치를 고려할 때, 지속적인 기술력 강화와 파트너십 확대 가능성이 존재한다.']
부정 요인 (2개): ['자산총액이 2조원 미만으로, 연결재무제표 의무 공시가 유예되어 있어 재무상태에 대한 불확실성이 존재한다.', '현금 창출 단위의 손상 가능성이 언급되어 있어, 향후 재무적 위험이 우려된다.']

기업코드: 1040.0
기업명: CJ
긍정 요인 (3개): ['CJ 그룹은 다양한 사업 부문을 운영하며, 특히 식품 및 엔터테인먼트 분야에서 강력한 시장 지위를 보유하고 있음.', 'CJ대한통운은 물류 및 신유